# Fairness Toolkit Demo (v0.1.0)

This notebook demonstrates metric computation, simple reporting, and optional MLflow logging for the Measurement Module.

In [ ]:
import numpy as np
import pandas as pd
from fairness_pipeline_dev_toolkit.metrics import FairnessAnalyzer
from fairness_pipeline_dev_toolkit.integration.reporting import to_markdown_report

print("Versions:")
import fairness_pipeline_dev_toolkit as fpt
print("fairness_pipeline_dev_toolkit:", getattr(fpt, "__version__", "dev"))
import sys
print("python:", sys.version)


## Load Sample Data
You can adjust this path to point at a real dataset later.

In [ ]:
import os
path = "tests/fixtures/e2e_sample.csv" if os.path.exists("tests/fixtures/e2e_sample.csv") else "dev_sample.csv"
df = pd.read_csv(path)
df.head()

## Compute Metrics
- `demographic_parity_difference` (classification)
- `equalized_odds_difference` (classification)
- `mae_parity_difference` (regression-style using `score` vs `y_true`)

In [ ]:
fa = FairnessAnalyzer(min_group_size=2, backend="native")
results = {}
results["demographic_parity_difference"] = fa.demographic_parity_difference(
    y_pred=df["y_pred"].to_numpy(), sensitive=df["group"].to_numpy()
)
results["equalized_odds_difference"] = fa.equalized_odds_difference(
    y_true=df["y_true"].to_numpy(), y_pred=df["y_pred"].to_numpy(), sensitive=df["group"].to_numpy()
)
results["mae_parity_difference"] = fa.mae_parity_difference(
    y_true=df["y_true"].to_numpy().astype(float),
    y_pred=df["score"].to_numpy().astype(float),
    sensitive=df["group"].to_numpy(),
)
results

## Markdown Report
A quick human-readable summary for PRs or artifacts.

In [ ]:
md = to_markdown_report(results, title="Fairness Demo Report")
print(md)

# Optional: save
with open("artifacts_demo_report.md", "w", encoding="utf-8") as f:
    f.write(md)
print("Saved artifacts_demo_report.md")